In [1]:
import mne
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader,Subset
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

C:\Users\matth\AppData\Local\Temp\ipykernel_33412\3426493753.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
class CONV(nn.Module):
    def __init__(self):
        super(CONV, self).__init__()
        # Goes over each channel horizontally (temporal)
        self.conv1 = nn.Conv2d(1, 16, (1, 128))

        self.batchnorm1 = nn.BatchNorm2d(16)
        # Goes over all 23 channels at the same time to find spatial patterns in the new temporal filters 
        self.conv2 = nn.Conv2d(16, 32, (23,1))

        self.batchnorm2 = nn.BatchNorm2d(32)
        # Squishes all of the channels down into a 1x(96 or smaller) output to extract the most important parts of the data
        self.pooling2 = nn.MaxPool2d((23,2),(1,2))

        self.conv3 = nn.Conv1d(32, 64, 1)

        self.batchnorm3 = nn.BatchNorm2d(64)

        self.pooling3 = nn.MaxPool1d((2,1), 2)

        self.f1 = nn.Linear(1 * 48 * 64, 1)

        self.dropout = nn.Dropout(0.3)
    def forward(self, x):
        # Layer 1
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = F.relu(x)
        x = self.dropout(x)

        # Layer 2
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        x = self.pooling2(x)
        x = self.dropout(x)

        # Layer 3
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = F.relu(x)
        x = self.pooling3(x)
        
        # Fully connected
        x = self.view(-1, 1 * 48 * 64)
        x = F.sigmoid(self.f1(x))
        return x

In [6]:
attempt = CONV()
print(attempt(torch.rand(1, 23,512)))

ValueError: expected 4D input (got 3D input)